# 使用 AKS online endpoint 部署机器学习模型

本教程将介绍如何使用在线端点部署机器学习模型，这样您就不必创建和管理底层基础结构。您将首先在本地机器上部署模型以调试任何错误，然后您将在 Azure 中部署和测试它。

有关详细信息，请参阅 [什么是 Azure 机器学习端点？](https://docs.microsoft.com/azure/machine-learning/concept-endpoints)。

## 部署端点（Deploy endpoints）

AzureML 提供三种类型的部署：

- 标准部署（standard deployment）: 适用于基本模型及其微调版本
- 在线端点（online endpoint）: 适用于生产环境
- 批量推理（batch endpoint）: 适用于批量推理


## 托管在线端点（Managed online endpoint）vs Kubernetes 在线端点（AKS）

托管联机终结点以方便的统包方式部署机器学习模型，是使用 Azure 机器学习联机终结点的推荐方式。 托管联机终结点在 Azure 中以一种可缩放的、完全托管的方式使用功能强大的 CPU 和 GPU 计算机。

如果你更倾向于使用 Kubernetes 来部署模型并提供端点服务，且你能够自如地管理基础设施需求，那么你可以使用 Kubernetes 在线端点。这些端点使你能够在任何经过完全配置和管理的 Kubernetes 集群上，利用 CPU 或 GPU 部署模型并提供在线端点服务。

下面的表格展示两者关键的区别：

|-|托管联机终结点|Kubernetes 联机终结点 (AKS v2)|
|---|---|---|  
|建议的用户|需要托管模型部署和增强式 MLOps 体验的用户|首选 Kubernetes 并可自行管理基础结构要求的用户|  
|**灵活预配**|托管式计算预配、更新、移除|用户负责|  
|**节点维护**|托管主机 OS 映像更新和安全强化|用户负责|  
|**群集大小调整（缩放）**|[托管式手动和自动缩放](https://learn.microsoft.com/zh-cn/azure/machine-learning/how-to-autoscale-endpoints?view=azureml-api-2)，支持额外节点预配|[手动和自动缩放](https://learn.microsoft.com/zh-cn/azure/machine-learning/how-to-kubernetes-inference-routing-azureml-fe?view=azureml-api-2#autoscaling)，支持在固定群集边界内缩放副本数|  
|**计算类型**|由服务管理|客户管理的 Kubernetes 群集|  
|**托管的标识**|[支持](https://learn.microsoft.com/zh-cn/azure/machine-learning/how-to-access-resources-from-endpoints-managed-identities?view=azureml-api-2)|支持|  
|**虚拟网络**|[通过托管网络隔离支持](https://learn.microsoft.com/zh-cn/azure/machine-learning/concept-secure-online-endpoint?view=azureml-api-2)|用户负责|  
|现成的监视和日志记录|[由 Azure Monitor 和 Log Analytics 提供支持](https://learn.microsoft.com/zh-cn/azure/machine-learning/how-to-monitor-online-endpoints?view=azureml-api-2)（包括终结点和部署的关键指标和日志表）|用户负责|  
|**使用 Application Insights（旧版）进行日志记录**|支持|支持|  
|成本视图|[细致到终结点/部署级别](https://learn.microsoft.com/zh-cn/azure/machine-learning/how-to-view-online-endpoints-costs?view=azureml-api-2)|群集级别|  
|成本适用对象|分配给部署的虚拟机 (VM)|分配给群集的 VM|  
|镜像流量|[支持](https://learn.microsoft.com/zh-cn/azure/machine-learning/how-to-safely-rollout-online-endpoints?view=azureml-api-2#test-the-deployment-with-mirrored-traffic)|不支持|  
|无代码部署|支持 [MLflow](https://learn.microsoft.com/zh-cn/azure/machine-learning/how-to-deploy-mlflow-models-online-endpoints?view=azureml-api-2) 和 [Triton](https://learn.microsoft.com/zh-cn/azure/machine-learning/how-to-deploy-with-triton?view=azureml-api-2) 模型|支持 [MLflow](https://learn.microsoft.com/zh-cn/azure/machine-learning/how-to-deploy-mlflow-models-online-endpoints?view=azureml-api-2) 和 [Triton](https://learn.microsoft.com/zh-cn/azure/machine-learning/how-to-deploy-with-triton?view=azureml-api-2) 模型|





## 部署的 VM 配额分配
对于托管联机终结点，Azure 机器学习会保留 20% 的计算资源，以便在某些 VM SKU 上执行升级。 如果你在某个部署中为这些 VM SKU 请求给定数量的实例，则为了避免收到错误，你必须有 ceil(1.2 * number of instances requested for deployment) * number of cores for the VM SKU 配额可用。 例如，如果在部署中请求 Standard_DS3_v2 VM（带有 4 个内核）的 10 个实例，则应该为 48 个内核 (12 instances * 4 cores) 提供可用配额。 此额外配额是为系统启动的操作（如 OS 升级和 VM 恢复）保留的，除非此类操作运行，否则不会产生费用。

某些 VM SKU 可以免除额外的配额预留。 要查看完整列表，请参阅[《托管联机终结点 SKU 列表》](https://learn.microsoft.com/zh-cn/azure/machine-learning/reference-managed-online-endpoints-vm-sku-list?view=azureml-api-2)。 若要查看使用情况和请求增加配额，请参阅在 Azure 门户中查看使用情况和配额。 若要查看运行托管联机终结点的成本，请参阅查看托管联机终结点的费用。

### 1. 设置先决条件

**要求**
- 对机器学习有基本了解
- 拥有具有活跃订阅的Azure账户 - [免费创建账户](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- 拥有带计算集群的Azure ML工作区 - [配置工作区](https://aka.ms/azureml-workspace-configuration)
- Python环境
- 已安装Azure机器学习Python SDK v2 - [安装说明](https://aka.ms/azureml-sdkv2-install) - 查看入门部分


* 连接到AzureML工作区。在[设置SDK认证](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk)了解更多。替换下面的 `<WORKSPACE_NAME>`、`<RESOURCE_GROUP>` 和 `<SUBSCRIPTION_ID>`。
* 连接到 `azureml` 系统注册表

# 1. 连接到 Azure Machine Learning 工作区

工作区是与同事协作创建机器学习工件和分组相关工作的场所。例如，实验、作业、数据集、模型、组件和推理端点。

## 1.1 导入需要的依赖

In [ ]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    KubernetesOnlineEndpoint,
    KubernetesOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities._deployment.resource_requirements_settings import (
    ResourceRequirementsSettings,
)
from azure.ai.ml.entities._deployment.container_resource_settings import (
    ResourceSettings,
)

## 1.2. 配置工作区，并获取实例

连接工作区需要这些参数 - subscription, 资源组名 和 工作区名。通过这些参数实例化 `MLClient，调用 SDK 需要授权，这里我们使用默认授权 [interactive authentication](https://docs.microsoft.com/python/api/azure-identity/azure.identity.interactivebrowsercredential?view=azure-python) 
更多设置参考 [here](https://docs.microsoft.com/python/api/azure-identity/azure.identity?view=azure-python).

- subscription 是订阅id，
- resource_group 是资源组名，同样从右上角菜单可以查到
- workspace 是工作区名

In [ ]:
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"

In [ ]:
# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

## 使用 local 端点进行本地测试

### 注意
* 本地部署需要保证 [Docker Engine](https://docs.docker.com/engine/install/) 已经安装好
* Docker 一般随系统启动，如果没有，参考 [troubleshoot Docker Engine](https://docs.docker.com/config/daemon/#start-the-daemon-manually).

# 2. 创建本地端点和部署

## 2.1 创建本地端点

本地部署的目的是在发布到 Azure 之前验证和调试配置。本地部署有下面这些限制：
* 本地端点 *不支持* 流量规则，授权和探针设置
* 本地端点支持每个端点一个部署

In [ ]:
# Creating a local endpoint
import datetime

local_endpoint_name = "local-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = KubernetesOnlineEndpoint(
    name=local_endpoint_name, description="this is a sample local endpoint"
)

In [ ]:
# Install docker package in the current Jupyter kernel
import sys

!{sys.executable} -m pip install docker

In [ ]:
ml_client.online_endpoints.begin_create_or_update(endpoint, local=True)

## 2.2 创建本地部署

这个示例包含了在在线端点上部署模型所需的所有文件。要部署一个模型，您必须拥有：

* 模型文件（或已在工作区中注册的模型的名称和版本）。在这个示例中，我们有一个用于回归的 scikit-learn 模型。
* 运行模型所需的代码。在这个例子中，我们有一个 score.py 文件。
* 模型运行的环境。您将看到，环境可以是带有 Conda 依赖项的 Docker 镜像，也可以是一个 Dockerfile。
* 用于指定实例类型和扩展能力的设置。

### 部署的关键方面

- `name` - 部署的名称。
- `endpoint_name` - 要在其下创建部署的端点名称。
- `model` - 用于部署的模型。这个值可以是工作区中现有版本化模型的引用，也可以是内联模型规范。
- `environment` - 用于部署的环境。这个值可以是工作区中现有版本化环境的引用，也可以是内联环境规范。
- `code_configuration` - 源代码和评分脚本的配置
    - `path` - 用于模型评分的源代码目录路径
    - `scoring_script` - 源代码目录中评分文件的相对路径
- `instance_type` - 用于部署的虚拟机大小。
- `instance_count` - 用于部署的实例数量
- `resources` - 部署的资源请求和限制。
    - `requests` - 一个部署实例被调度所需的最小资源请求。对于所有部署，都应该指定 CPU 和内存的请求。
    - `limits` - （可选）一个部署实例可以使用的最大资源。当限制值缺失时，将使用实例类型中的设置。对于 GPU 工作负载，还应该指定 CPU 和内存的限制。如果为部署指定 GPU，应该在 limits 字段中指定，或者在 requests 和 limits 中都指定相同的值。


In [ ]:
model = Model(path="../model-1/model/sklearn_regression_model.pkl")
env = Environment(
    conda_file="../model-1/environment/conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
)

blue_deployment = KubernetesOnlineDeployment(
    name="blue",
    endpoint_name=local_endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="../model-1/onlinescoring", scoring_script="score.py"
    ),
    instance_count=1,
    resources=ResourceRequirementsSettings(
        requests=ResourceSettings(
            cpu="100m",
            memory="0.5Gi",
        ),
    ),
)

In [ ]:
ml_client.online_deployments.begin_create_or_update(
    deployment=blue_deployment, local=True
)

# 3. 校验本地部署成功创建

## 3.1 检查下面一些状态来确认模型部署无误

In [ ]:
ml_client.online_endpoints.get(name=local_endpoint_name, local=True)

## 3.2 获取 Logs

In [ ]:
ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=local_endpoint_name, local=True, lines=50
)

让我帮您翻译这段内容：

## 3.3 调用本地端点
通过使用便捷命令 invoke 并传入存储在 JSON 文件中的查询参数来调用端点，对模型进行评分。

这段文字描述了一个操作步骤，主要是关于如何调用本地端点来测试模型。它使用了两个关键部分：
1. 使用 `invoke` 命令
2. 从 JSON 文件中读取查询参数

这是一个典型的机器学习模型部署和测试场景，通常用于验证模型在生产环境中的表现。

In [ ]:
ml_client.online_endpoints.invoke(
    endpoint_name=local_endpoint_name,
    request_file="../model-1/sample-request.json",
    local=True,
)

# 4. 为机器学习配置 Kubernetes 集群
接下来，配置 Azure Kubernetes Service (AKS) 和启用 Azure Arc 的 Kubernetes 集群，用于机器学习推理工作负载。
以下步骤有一些先决条件，您可以在[这里](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-attach-arc-kubernetes)查看。

## 4.1 将现有 Kubernetes 集群连接到 Azure Arc
对于 [AKS 集群](https://docs.microsoft.com/en-us/azure/aks/kubernetes-walkthrough)来说，此步骤是可选的。
按照此[指南](https://docs.microsoft.com/en-us/azure/azure-arc/kubernetes/quickstart-connect-cluster)连接 Kubernetes 集群。

## 4.2 部署 Azure Machine Learning 扩展
根据您的网络设置、Kubernetes 发行版变体以及 Kubernetes 集群的托管位置（本地或云端），选择以下选项之一来部署 Azure Machine Learning 扩展，并在您的 Kubernetes 集群上启用推理工作负载。
按照此[指南](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-attach-arc-kubernetes?tabs=studio#inferencing)操作。

## 4.3 附加 Arc 集群
您可以使用 Studio、Python SDK 和 CLI 将 Arc 集群附加到 Machine Learning 工作区。
以下代码显示了附加 AKS 的示例，其计算类型为 managedClusters。对于 Arc 连接的集群，应该使用 connectedClusters。
有关更多详细信息，请按照此[指南](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-attach-arc-kubernetes?tabs=studio#attach-arc-cluster)操作。

这段内容主要描述了如何配置和设置 Kubernetes 集群以支持机器学习工作负载，包括连接到 Azure Arc、部署必要的扩展以及将集群附加到机器学习工作区的具体步骤。


In [ ]:
from azure.ai.ml import load_compute

compute_name = "<COMPUTE_NAME>"

# for arc connected cluster, the resource_id should be something like '/subscriptions/<SUBSCRIPTION_ID>/resourceGroups/<RESOURCE_GROUP>/providers/Microsoft.ContainerService/connectedClusters/<CLUSTER_NAME>''
compute_params = [
    {"name": compute_name},
    {"type": "kubernetes"},
    {
        "resource_id": "/subscriptions/<SUBSCRIPTION_ID>/resourceGroups/<RESOURCE_GROUP>/providers/Microsoft.ContainerService/managedClusters/<CLUSTER_NAME>"
    },
]
k8s_compute = load_compute(source=None, params_override=compute_params)

# !!!bug https://msdata.visualstudio.com/Vienna/_workitems/edit/1740311
compute_list = {c.name: c.type for c in ml_client.compute.list()}

if compute_name not in compute_list or compute_list[compute_name] != "kubernetes":
    ml_client.begin_create_or_update(k8s_compute).result()
else:
    print("Compute already exists")

# 5. 将在线端点部署到 Azure
接下来，将您的在线端点部署到 Azure。

## 5.1 配置在线端点
`endpoint_name`：端点的名称。

`auth_mode`：使用 `key` 进行基于密钥的身份验证。使用 `aml_token` 进行基于 Azure Machine Learning 令牌的身份验证。`key` 不会过期，但 `aml_token` 会过期。

您还可以选择为端点添加描述和标签。

这段内容描述了如何配置 Azure 在线端点的基本设置，包括端点命名和身份验证方式的选择。其中特别说明了两种不同的身份验证模式及其特点。


In [ ]:
# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime

online_endpoint_name = "k8s-endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = KubernetesOnlineEndpoint(
    name=online_endpoint_name,
    compute="<COMPUTE_NAME>",
    description="this is a sample online endpoint",
    auth_mode="key",
    tags={"foo": "bar"},
)

## 5.2 创建端点

使用之前创建的 `MLClient`，我们现在将在工作区中创建端点。此命令将启动端点创建过程，并在端点创建继续进行的同时返回确认响应。

这段内容描述了如何使用 MLClient 来创建端点的过程，说明了命令执行后会立即返回响应，而端点的实际创建会在后台继续进行。


In [ ]:
ml_client.begin_create_or_update(endpoint).result()

## 5.3 配置在线部署

部署是一组用于托管执行实际推理的模型所需的资源。我们将使用 `KubernetesOnlineDeployment` 类为我们的端点创建一个部署。

这段内容解释了部署的概念，并说明了将使用 KubernetesOnlineDeployment 类来创建部署。部署包含了运行模型推理所需的所有必要资源。


In [ ]:
model = Model(path="../model-1/model/sklearn_regression_model.pkl")
env = Environment(
    conda_file="../model-1/environment/conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
)

blue_deployment = KubernetesOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="../model-1/onlinescoring", scoring_script="score.py"
    ),
    instance_count=1,
    resources=ResourceRequirementsSettings(
        requests=ResourceSettings(
            cpu="100m",
            memory="0.5Gi",
        ),
    ),
)

## 5.4 创建部署

使用之前创建的 `MLClient`，我们现在将在工作区中创建部署。此命令将启动部署创建过程，并在部署创建继续进行的同时返回确认响应。

这段内容描述了如何使用 MLClient 来创建部署的过程，说明了命令执行后会立即返回响应，而部署的实际创建会在后台继续进行。这与之前创建端点的过程类似，都是异步操作。


In [ ]:
ml_client.begin_create_or_update(blue_deployment).result()

In [ ]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
ml_client.begin_create_or_update(endpoint).result()

# 6. 使用样本数据测试端点
使用之前创建的 `MLClient`，我们将获取端点的句柄。可以使用 `invoke` 命令调用端点，该命令需要以下参数：
- `endpoint_name` - 端点名称
- `request_file` - 包含请求数据的文件
- `deployment_name` - 端点中要测试的特定部署的名称

我们将使用 [json](./model-1/sample-request.json) 文件发送一个示例请求。

这段内容描述了如何使用 MLClient 来测试已部署的端点，并详细说明了调用端点所需的参数，以及将使用 JSON 文件来发送测试请求。


In [ ]:
# test the blue deployment with some sample data
# comment this out as cluster under dev subscription can't be accessed from public internet.
# ml_client.online_endpoints.invoke(
#    endpoint_name=online_endpoint_name,
#    deployment_name='blue',
#    request_file='../model-1/sample-request.json')

# 7. 管理端点和部署

## 7.1 获取端点的详细信息

In [ ]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

## 7.2 获取日志
获取部署的日志并根据需要进行验证

In [ ]:
ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=online_endpoint_name, lines=50
)

# 8. 删除端点


In [ ]:
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)